# 5. Distributiegrafiek
Historische ontwikkelingen kunnen ook op een andere manier worden geanalyseerd. Appplicaties zoals de [Google Books Ngram viewer](https://books.google.com/ngrams) maken gebruik van distributie-grafieken waarin woordfrequenties per jaar worden berekend. De onderstaande ocde doet iets vergelikbaar voor het corpus van "Soldaat in Indonesië". Zoals te zien is in de eerste regels van de onderstaande cel wordt er eerst een lljst van woorden opgegeven. Deze woorden worden allemaal vastgelegd onder de naam `search_terms`. De code berekent vervolgens per jaar hoe vaak deze woorden zijn gebruikt. De resultaten worden vastgelegd in een CSV-bestand waarin de relatieve frequenties (het aantal voorkomens va deze woorden, gedeeld door het totaal aantal woorden in dat jaar) worden vastgelegd. 

In [ ]:
from kitlvTdm import *
import os
from os.path import join
import re
import pandas as pd

In [ ]:
search_terms = '''
gestoken
vuur
schieten
gevangen
doodgeschoten
schoten
neergeschoten
gevangenen
patrouille
gevangene
dood
geschoten
soldaten
doden
bloed
geweer
gedood
wapens
beschoten
schoot
kogel
gewonden
geslagen
vermoord
verhoord
zuiveringsactie
hinderlaag
slachtoffers
executie
patrouilles
wraak
inlichtingendienst
extremisten
kogels
woede
moord
mortieren
krijgsraad
vuurgevecht
incident
kampement
gewonde
handgranaat
vluchtende
gevecht
geëxecuteerd
pistool
gewond
oorlog
fik
tegenstanders
krijgsgevangenen
strijd
mitrailleur
spion
platgebrand
kapot
afgebrand
zuivering
bajonet
salvo
mishandeld
omsingeld
granaten
gevangengenomen
bataljon
vernietigd
represaille
beschieting
tegenpartij
terroristen
plunderaars
doodschieten
brandende
hardhandig
moordenaars
genadeschot
pijn
revolver
gemarteld
gefusilleerd
mes
wapen
aangevallen
neerschieten
gezuiverd
brandde
plunderen
vijanden
beschietingen
gedreigd
geplunderd
slaag
executeren
aanslag
geschreeuw
handgranaten
afschuwelijke
gebombardeerd
verminkt
gemene
burgerslachtoffers
plundering
plunderingen
krijgsgevangene
moorden
excessief
geweldsexcessen
mishandeling
gevechten
sadistische
vuurstoot
weerloze
martelen
executies
doodgeslagen
schreeuwde
schietend
onthoofd
represailles
fosforgranaten
verminkte
gruwelijk
nekschot
tegenaanval
overval
verwond
moordenaar
bewapend
opblazen
executiepeloton
verkracht
vuurstoten
vuurwapens
vergelding
beestachtige
doodstraf
trekker
stervende
neergeknald
platbranden
artillerie
bedreiging
slachting
verwondingen
vreselijke
ontgelden
gummiknuppel
knallen
gevaarlijk
slachtoffer
marteling
beestachtig
platgeschoten
lijden
uitgemoord
schietpartij
bloedbad
mortiergranaten
misdadigers
massagraf
gruwelijke
afgehakt
kapmes
bombardement
schokkende
zuiveringsopdracht
'''

In [ ]:
import re

# splits de lijst hierboven op in losse woorden
search_terms = search_terms.strip()
terms_list = re.split( r'\n' , search_terms.lower() )

# In deze dictionaries worden resultaten bijgehouden
frequency_year = dict()
word_count_year  = dict()
corpusFreq = dict()

min_year = 1930
max_year = 1960

path = 'corpus'


for file in os.listdir( path ):
    if re.search( '[.]txt$' , file ):
        print( f"Reading { file } ..." )
        year = showYear(file)
        if year != "" and len(year) == 4:
            year = int(year)
            if year >= min_year and year <= max_year:
                
                year = int(year)
                freq = calculateWordFrequencies( join( path , file ) )
                freq = removeStopwords( freq )

                tokens = numberOfTokens( join( path , file ) )

                # pak de word count van het jaar als die al bestaat, anders 0.
                word_count_year[ year ] = word_count_year.get( year , 0 ) + tokens

                # voor iedere term in de zoeklijst, voeg de frequen
                for term in terms_list:
                    if term in freq:
                        frequency_year[year] = frequency_year.get( year , 0 ) + freq[ term ]
                print(frequency_year)

            
df = pd.DataFrame(columns = ["year", "frequency"])


for year in sorted( frequency_year  ):
    df = df.append({"year" : year, "frequency" : frequency_year[ year ] / word_count_year[year]}, ignore_index = True)
    print( '{} times in {}.'.format( frequency_year[ year ] , year ) )





De frequenties kunnen hierna worden gevisualiseerd met de onderstaande code.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


# Dit is de stijl van de plot
plt.style.use('ggplot')

# Door figsize aan te passen kun je de groote van de tabel veranderen
fig = plt.figure( figsize = ( 15, 4))
ax = plt.axes()

# Plot year vs frequency
ax.scatter( df['year'] , df['frequency'] , color = '#930d08' , linestyle = 'solid')

# Bepaal de labels bij de assen
ax.set_xlabel('Years')
ax.set_ylabel('Word frequency')

# Zet de x limiet
ax.set_xlim( df['year'].min() , df['year'].max() )

# Bepaal de titel
ax.set_title( 'frequencies per year')

# Hier wordt een "best fit" lijn berekent en over de losse punten geplot
linear_model = np.polyfit(df["year"],df["frequency"], 1)
linear_model_fn = np.poly1d(linear_model)
x_s = np.arange(df["year"].min(), df["year"].max())
plt.plot(x_s,linear_model_fn(x_s),color="green")

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


# Dit is de stijl van de plot
plt.style.use('ggplot')

# Door figsize aan te passen kun je de groote van de tabel veranderen
fig = plt.figure( figsize = ( 15, 4))
ax = plt.axes()

# Plot year vs frequency
ax.plot( df['year'] , df['frequency'] , color = '#930d08' , linestyle = 'solid')

# Bepaal de labels bij de assen
ax.set_xlabel('Years')
ax.set_ylabel('Word frequency')

# Zet de x limiet
ax.set_xlim( df['year'].min() , df['year'].max() )

# Bepaal de titel
ax.set_title( 'frequencies per year')

# Hier wordt een "best fit" lijn berekent en over de losse punten geplot
linear_model = np.polyfit(df["year"],df["frequency"], 1)
linear_model_fn = np.poly1d(linear_model)
x_s = np.arange(df["year"].min(), df["year"].max())
plt.plot(x_s,linear_model_fn(x_s),color="green")

plt.show()

**Oefening 6: Pas de waarde van de variabele `search_terms` aan. Geef een aantal woorden op die van belang zijn voor jouw onderzoek. Probeer hierna een distributiegrafiek te maken die de historische ontwikkelingen in het gebruik van deze termen kan verduidelijken.**